In [1]:
#Pytorch
import torch
import torch.nn as nn
import torch.autograd as autograd
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
#Python Libs
import numpy as np
from time import time
import datetime
#Implementations
from model.agree_re import VARGR_AGREE3
from config_CA import Config
from dataset_Meetup import GDataset
from batch_test import *

In [2]:
WU_point = 20
def training(model, train_loader, epoch_id, config):
    # user trainning
    t1 = time()
    model.train()
    learning_rates = config.lr
    lr = learning_rates[0] #if epoch_id<WU_point else 0.01
    beta = 0.0 if epoch_id<WU_point else 1.0
    beta2 = 0.0 if epoch_id<2*WU_point else 1.0
    # optimizer
    optimizer = optim.Adam(model.parameters(), lr=lr)

    losses = []

    *_, last = train_loader

    for batch_id, (g, pi_ni) in enumerate(train_loader):
        # Data Load
        group_input = g
        pos_item_input = pi_ni[:, 0]
        neg_item_input = pi_ni[:, 1]

        pos_prediction, pos_prediction2, dkl = model(group_input, pos_item_input, is_training=True)
        neg_prediction, neg_prediction2, _ = model(group_input, neg_item_input, is_training=True)

        # Zero_grad
        model.zero_grad()
        # Loss
        eps=1e-7
        loss = (1-beta2)*(torch.mean(-torch.log(pos_prediction+eps)-torch.log(1.0-neg_prediction+eps))+beta*(dkl))+beta2*(torch.mean(-torch.log(pos_prediction2+eps)-torch.log(1.0-neg_prediction2+eps)))
        
        # record loss history
        #print("batch_id: " + str(batch_id) + " loss: " + str(loss.item()))
        if not torch.isinf(loss.data) and not torch.isnan(loss.data):
            losses.append(float(loss.item()))
        # Backward
        loss.backward()
        optimizer.step()

    print('Iteration %d,\tloss: [%.4f], time: [%.1fs]' % (epoch_id, np.mean(np.array(losses)), time() - t1))


In [3]:
def evaluation(model, groups_to_test, Ks, trainRatings, num_items):
    model.eval()
    t2 = time()
    ret = test(model, groups_to_test, Ks, trainRatings, num_items) #See batch_test

    print('\t Evaluation done [%.1f s]' % (time() - t2))
    for i, k in enumerate(Ks):
        print('\t\t @%d: HR = %.4f, NDCG = %.4f, Rec = %.4f' % (k, ret['hit_ratio'][i], ret['ndcg'][i], ret['recall'][i]))
    return ret

In [4]:
config = Config()

meetup.ca 80 256


In [5]:
dataset = GDataset(config.user_dataset, config.group_dataset, config.user_in_group_path, 1)
num_groups, num_users, num_items = dataset.num_groups, dataset.num_users, dataset.num_items
print("num_groups: "+str(num_groups)+" num_users: "+str(num_users)+" num_items: "+str(num_items))
gu_dict = dataset.gu_dict

num_groups: 607 num_users: 56878 num_items: 1490


In [6]:
test_warm = dataset.load_rating_file_as_dict(config.group_dataset + ".test.rating_warm")
test_cold = dataset.load_rating_file_as_dict(config.group_dataset + ".test.rating_cold")
#ret = evaluation(agree, test_warm, config.topK[:2], dataset.group_trainRatings, dataset.num_items)
#ret = evaluation(agree, test_cold, config.topK[:2], dataset.group_trainRatings, dataset.num_items)

In [7]:
agree = VARGR_AGREE3(num_users, num_items, num_groups, config.embedding_size, gu_dict, config.drop_ratio).cuda()
best_checkpoint = -1.0
best_weights_path = None
for num_negatives in config.num_negatives:
    dataset.num_negatives = num_negatives
    print("AGREE: embedding size %d, run Iteration: %d, #neg: %d" %(config.embedding_size, config.epoch, num_negatives))
    # train the model
    now = datetime.datetime.now().strftime("%Y-%m-%d")

    for epoch in range(config.epoch): 
        training(agree, dataset.get_group_dataloader(config.batch_size), epoch, config)
        

        # Evaluation
        if epoch > WU_point:

            #ret = evaluation(agree, dataset.group_testRatings, config.topK[:2], dataset.group_trainRatings, dataset.num_items)
            ret = evaluation(agree, test_warm, config.topK[:2], dataset.group_trainRatings, dataset.num_items)
            cur_checkpoint = ret['ndcg'][1]
            #ret = evaluation(agree, test_cold, config.topK[:2], dataset.group_trainRatings, dataset.num_items)
            #cur_checkpoint += ret['hit_ratio'][1]
            #cur_checkpoint = cur_checkpoint/2
            current_weights_path = 'weights/VARGR_AGREE3_'+str(config.dataset)+"_"+str(config.embedding_size)+"_"+str(config.lr[0])+'_'+str(num_negatives)+'_'+str(epoch)
            torch.save(agree.state_dict(), current_weights_path)
            if best_checkpoint <= cur_checkpoint:
                best_weights_path = current_weights_path
                best_checkpoint = cur_checkpoint

AGREE: embedding size 32, run Iteration: 80, #neg: 4
Iteration 0,	loss: [1.1653], time: [92.1s]
Iteration 1,	loss: [0.8119], time: [91.1s]
Iteration 2,	loss: [0.4782], time: [91.0s]
Iteration 3,	loss: [0.3105], time: [91.7s]
Iteration 4,	loss: [0.2197], time: [91.9s]
Iteration 5,	loss: [0.1551], time: [100.9s]
Iteration 6,	loss: [0.1325], time: [80.3s]
Iteration 7,	loss: [0.1075], time: [64.2s]
Iteration 8,	loss: [0.0923], time: [69.1s]
Iteration 9,	loss: [0.0882], time: [69.8s]
Iteration 10,	loss: [0.0822], time: [64.0s]
Iteration 11,	loss: [0.0773], time: [65.3s]
Iteration 12,	loss: [0.0774], time: [65.7s]
Iteration 13,	loss: [0.0888], time: [64.8s]
Iteration 14,	loss: [0.0878], time: [64.3s]
Iteration 15,	loss: [0.0827], time: [65.2s]
Iteration 16,	loss: [0.0872], time: [65.3s]
Iteration 17,	loss: [0.0928], time: [65.4s]
Iteration 18,	loss: [0.0878], time: [74.3s]
Iteration 19,	loss: [0.0951], time: [82.8s]
Iteration 20,	loss: [59.7758], time: [81.1s]
Iteration 21,	loss: [29.2283], 

In [8]:
agree = VARGR_AGREE3(num_users, num_items, num_groups, config.embedding_size, gu_dict, config.drop_ratio).cuda()
agree.load_state_dict(torch.load(best_weights_path))
print(best_weights_path)

weights/VARGR_AGREE3_meetup.ca_32_0.01_4_43


In [9]:
print('total')
ret = evaluation(agree, dataset.group_testRatings, config.topK[:2], dataset.group_trainRatings, dataset.num_items)
print('cold')
ret = evaluation(agree, test_cold, config.topK[:2], dataset.group_trainRatings, dataset.num_items)
print('warm')
ret = evaluation(agree, test_warm, config.topK[:2], dataset.group_trainRatings, dataset.num_items)   

total
	 Evaluation done [238.1 s]
		 @5: HR = 0.3333, NDCG = 0.1952, Rec = 0.2131
		 @10: HR = 0.5115, NDCG = 0.2700, Rec = 0.3669
cold
	 Evaluation done [59.9 s]
		 @5: HR = 0.2273, NDCG = 0.1394, Rec = 0.0985
		 @10: HR = 0.3864, NDCG = 0.1932, Rec = 0.1913
warm
	 Evaluation done [179.5 s]
		 @5: HR = 0.4154, NDCG = 0.2390, Rec = 0.2885
		 @10: HR = 0.5538, NDCG = 0.3057, Rec = 0.4282
